In [ ]:
from board import Color
import os
import os.path as path
import numpy as np
DATA_ROOT = '/mnt/share/aai_kikura'

## データ読み込み

In [ ]:
kifu_list = os.listdir(path.join(DATA_ROOT, './kifu'))
kifu_list = list(map(lambda x: path.join(DATA_ROOT, './kifu', x), kifu_list))

## パース
パース途中にエラーが起きたらその対局データは捨てる

In [ ]:
SPACE = ' '
c2num = {c:i for i, c in enumerate('ABCDEFGH')}

def parse(line):
    try:
        cursor = 0
        color = [Color.Black]
        plies = []

        def append_ply(y, x, color=color):
            plies.append((color[0], np.int32(int(y)*8+int(x)-1)))
            color[0] = Color.Black if color[0] == Color.White else Color.White

        while True:
            y = line[cursor]
            cursor += 1

            if y == SPACE:  
                if line[cursor] not in c2num:
                    # EoK
                    meta = line[cursor-1:]
                    return {
                        'plies': plies,
                        'meta': meta
                    }

                else: 
                    # PASS
                    append_ply(0, -1)
                    continue

            y  = c2num[y]

            x = line[cursor]
            cursor += 1

            append_ply(y, x)

    except Exception:
        return None
        
def parse_file(file_path):
    with open(file_path, 'r') as f:
        texts = f.read()
        
    data = tuple([parse(line) for line in texts.strip().split('\n')])
    data = [d for d in data if d is not None]    
    return data

data_all = []

for i, kifu in enumerate(kifu_list):
    print(i, kifu)
    data_all.extend(parse_file(kifu))

## 書き出し

In [ ]:
import pickle

with file.open(path.join(DATA_ROOT, './data.pkl'), 'wb') as f:
    pickle.dump(data_all, f)